# Running Cicada Programs

Because a Cicada "program" is actually multiple copies of a single program running simultaneously on multiple machines, bootstrapping the entire process can be tricky.  It would quickly become annoying if you had to execute five separate command lines every time you wanted to run a program with five players, especially during development and debugging!

To streamline the process, Cicada provides tools to make the process simpler during each of the following use-cases:

In [3]:
# my-mpc.py
import cicada.communicator

@cicada.communicator.NNGCommunicator.run(world_size=5)
def main(communicator):
    # Your program here
    pass
    
main()

[None, None, None]